In [1]:
import numpy as np 

import sys
import matplotlib
import lsssys
import twopoint
import fitsio as fio
import healpy as hp 


Reading config file
Working with Y3


In [2]:
#paths and config

nside = 512
extra_label = ''

ngal_dir = '/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample_0.5.1_wide_0.9binning_zmax0.95/ngal_maps/'
ngal_file_temp = ngal_dir + 'y3_gold_2.2.1_wide_sofcol_run_redmapper_v0.5.1_combined_hd3_hl2_galmap_nside512_bin{ibin}_zmin{zmin}_zmax{zmax}.fits.gz'

f_map_dir = '/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample_0.5.1_wide_0.9binning_zmax0.95/weight_maps/'

#labels = [ 'fid_pca_107', 'fid_pca_108', 'enet_pca_108', 'enet_pca_107', 'enet_pca_108_hii99_cd99', 'enet_std_107',  'fid_std' , 'noweights', ]
#labels = [ 'enet_pca_108', 'enet_pca_108_hii99_cd99', 'enet_std_107', 'fid_pca_108', 'fid_std' ]
#labels = [ 'enet_pca_108', 'enet_pca_108_hii99_cd99', 'enet_std_107', ]
#labels = [ 'fid_std' ]
#labels = [ 'fid_pca_108', 'fid_std']
#labels = [ 'enet_pca_107' ]
#labels = [ 'fid_pca_107', 'noweights']
labels = [ '', ]

f_map_file_dict = {
    'enet_pca_108':           f_map_dir + 'enet_pca_108/enet_Fest_map_Nbase108_512_izbin{ibin}_minfrac0.1_sqrt_pca.fits.gz', 
    'enet_pca_108_hii99_cd99':f_map_dir + 'enet_pca_108_hii99_cd99/enet_Fest_map_Nbase108_512_izbin{ibin}_minfrac0.1_sqrt_pca_hii99.9_cd99.9.fits.gz', 
    'enet_std_107':           f_map_dir + 'enet_std_107/enet_Fest_map_Nbase107_512_izbin{ibin}_minfrac0.1_sqrt.fits.gz',
    'enet_std_108':           f_map_dir + 'enet_std_108/enet_Fest_map_Nbase108_512_izbin{ibin}_minfrac0.1.fits.gz',
    'enet_pca_50':            f_map_dir + '',
}

w_map_file_dict = {
    'fid_pca_107': f_map_dir + 'fid_pca_107/w_map_bin{ibin}_nside4096_nbins1d_10_2sig_v2.0_nside512.fits.gz',
    'fid_pca_108': f_map_dir + 'fid_pca_108/w_map_bin{ibin}_nside4096_nbins1d_10_2.0sig_v2.0_nside512.fits.gz', 
    'fid_std':     f_map_dir + 'fid_std/w_map_bin{ibin}_nside4096_nbins1d_10_2.0signside512.fits.gz',
}

binedges = [0.15, 0.35, 0.50, 0.65, 0.80, 0.90]

nthetabins = 20
thetamax = 250./60.
thetamin = 2.5/60.
bin_slop = 0.05
num_threads = 1

angle_edges = np.logspace(np.log10(thetamin*60.),np.log10(thetamin*60.), 21)


In [3]:
def load_galmap_w(ibin, label):
    ngal_data = fio.read(ngal_file_temp.format(ibin=ibin+1, zmin=binedges[ibin], zmax=binedges[ibin+1]) )
    data = np.ones(hp.nside2npix(nside))*hp.UNSEEN
    data[ngal_data['HPIX']] = ngal_data['VALUE']
    mask = np.ones(hp.nside2npix(nside)).astype('bool')
    mask[ngal_data['HPIX']] = False
    frac = np.zeros(hp.nside2npix(nside))
    frac[ngal_data['HPIX']] = ngal_data['FRACDET']
    galmap_w = lsssys.Map()
    galmap_w.adddata(data, mask, frac)
    if 'enet' in label:
        f_map_temp =  f_map_file_dict[label]
        f = lsssys.SysMap(f_map_temp.format(ibin=ibin) )
        if f.nside != galmap_w.nside:
            print('f map nside {0}, galmap_w nside {1}'.format(f.nside, galmap_w.nside))
            print('up/de-grading f')
            f.degrade(galmap_w.nside)
        assert (f.data[~galmap_w.mask] == hp.UNSEEN).any() == False
        galmap_w.data[~galmap_w.mask] = galmap_w.data[~galmap_w.mask]/(1. + f.data[~galmap_w.mask]) 
    elif 'fid' in label:
        w_map_temp = w_map_file_dict[label]
        w_map = lsssys.SysMap(w_map_temp.format(ibin=ibin), systnside=nside)
        if w_map.nside != galmap_w.nside:
            print(w_map.nside, galmap_w.nside)
            w_map.degrade(galmap_w.nside, weightedmean=True)
        assert (w_map.data[~galmap_w.mask] == hp.UNSEEN).any() == False
        galmap_w.data[~galmap_w.mask] = galmap_w.data[~galmap_w.mask]*(w_map.data[~galmap_w.mask]) 
    elif label == 'noweights':
        pass
    else:
        raise RuntimeError('enet or fid not in label')
        
    return galmap_w

In [14]:
#%%capture

#compute clustering
nbins = 5


for label in labels:
    w_dict = {}
    for ibin in range(nbins):
        try:
            galmap_w = load_galmap_w(ibin, label)
        except IOError:
            print('couldnt find', ibin)
            continue

        theta, wtheta = lsssys.corr2pt(galmap_w, galmap_w, 
          nthetabins, thetamax, thetamin, 
          bin_slop=bin_slop, num_threads=num_threads, bin_type=None, 
          delta_input=False, w1=None, w2=None, 
          scale1=1./galmap_w.fracdet, scale2=1./galmap_w.fracdet, 
          return_var=False, 
          returncorr=False, jointmask=None, 
          fracweights=True, fracweights2=True, 
          weights=None, weights2=None)
        w_dict['theta_{0}_{0}'.format(ibin)] = theta
        w_dict[ibin,ibin] = wtheta

    w_dict['angle_min'] = angle_edges[:-1]
    w_dict['angle_max'] = angle_edges[1:]

    spectrum = lsssys.corrdict_2_spectrumtype(w_dict, autoonly=True, name='wtheta', 
                kernel1='nz_lens', kernel2='nz_lens',)

    tp = twopoint.TwoPointFile([spectrum], kernels=None, windows={}, covmat_info=None)
    tp.to_fits('wtheta_redmagic_y3_data_bs{bin_slop}_{label}{extra_label}.fits'.format(
        bin_slop = bin_slop,
        label=label,
        extra_label=extra_label), overwrite=True)



/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample_0.5.1_wide_0.9binning_zmax0.95/weight_maps/fid_pca_107/w_map_bin0_nside4096_nbins1d_10_2sig_v2.0_nside512.fits.gz
.fits in syst name, forcing filename load
NSIDE is ok, but it is not healpy format
colnames = ['HPIX', 'VALUE']
Assuming celestial coordinates. If the coordinates are galactic, use gal2eq()


/Users/jackelvinpoole/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in divide


corr2pt using bin_slop = 0.05
corr2pt: auto
corr2pt took 16.3410561085s
/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample_0.5.1_wide_0.9binning_zmax0.95/weight_maps/fid_pca_107/w_map_bin1_nside4096_nbins1d_10_2sig_v2.0_nside512.fits.gz
.fits in syst name, forcing filename load
NSIDE is ok, but it is not healpy format
colnames = ['HPIX', 'VALUE']
Assuming celestial coordinates. If the coordinates are galactic, use gal2eq()
corr2pt using bin_slop = 0.05
corr2pt: auto
corr2pt took 16.0213091373s
/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample_0.5.1_wide_0.9binning_zmax0.95/weight_maps/fid_pca_107/w_map_bin2_nside4096_nbins1d_10_2sig_v2.0_nside512.fits.gz
.fits in syst name, forcing filename load
NSIDE is ok, but it is not healpy format
colnames = ['HPIX', 'VALUE']
Assuming celestial coordinates. If the coordinates are galactic, use gal2eq()
corr2pt using bin_slop = 0.05
corr2pt: auto
corr2pt took 16.1050601006s
/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample

# nside = 4096

In [6]:
nside = 4096
extra_label='nside4096'

ngal_dir = '/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample_0.5.1_wide_0.9binning_zmax0.95/ngal_maps/'
ngal_file_temp = ngal_dir + 'y3_gold_2.2.1_wide_sofcol_run_redmapper_v0.5.1_combined_hd3_hl2_galmap_nside4096_bin{ibin}_zmin{zmin}_zmax{zmax}.fits.gz'

w_map_file_dict = {
    'fid_pca_107': f_map_dir + 'fid_pca_107/w_map_bin{ibin}_nside4096_nbins1d_10_2sig_v2.0.fits.gz',
    'fid_pca_108': f_map_dir + 'fid_pca_108/w_map_bin{ibin}_nside4096_nbins1d_10_2.0sig_v2.0.fits.gz', 
    'fid_std':     f_map_dir + 'fid_std/w_map_bin{ibin}_nside4096_nbins1d_10_2.0sig.fits.gz',
}

#labels = [ 'fid_pca_108', 'enet_std_107']
#labels = [ 'fid_pca_107']
#labels = [ 'noweights', ]
#labels = [ 'enet_pca_108_hii99_cd99'  ]
#labels = [ 'enet_std_108', 'enet_pca_108' ]
labels = [ 'fid_std',  ]


In [ ]:

#compute clustering
nbins = 5
bin_slop = 0.1

for label in labels:
    w_dict = {}
    for ibin in range(nbins):
        try:
            galmap_w = load_galmap_w(ibin, label)
        except IOError:
            print('couldnt find', ibin)
            continue

        theta, wtheta = lsssys.corr2pt(galmap_w, galmap_w, 
          nthetabins, thetamax, thetamin, 
          bin_slop=bin_slop, num_threads=num_threads, bin_type=None, 
          delta_input=False, w1=None, w2=None, 
          scale1=1./galmap_w.fracdet, scale2=1./galmap_w.fracdet, 
          return_var=False, 
          returncorr=False, jointmask=None, 
          fracweights=True, fracweights2=True, 
          weights=None, weights2=None)
        w_dict['theta_{0}_{0}'.format(ibin)] = theta
        w_dict[ibin,ibin] = wtheta

    w_dict['angle_min'] = angle_edges[:-1]
    w_dict['angle_max'] = angle_edges[1:]

    spectrum = lsssys.corrdict_2_spectrumtype(w_dict, autoonly=True, name='wtheta', 
                kernel1='nz_lens', kernel2='nz_lens',)

    tp = twopoint.TwoPointFile([spectrum], kernels=None, windows={}, covmat_info=None)
    tp.to_fits('wtheta_redmagic_y3_data_bs{bin_slop}_{label}{extra_label}.fits'.format(
        bin_slop = bin_slop,
        label=label,
        extra_label=extra_label), overwrite=True)



/Users/jackelvinpoole/DES/cats/y3/redmagic/combined_sample_0.5.1_wide_0.9binning_zmax0.95/weight_maps/fid_std/w_map_bin0_nside4096_nbins1d_10_2.0sig.fits.gz
.fits in syst name, forcing filename load


/Users/jackelvinpoole/anaconda2/lib/python2.7/site-packages/healpy/fitsfunc.py:287: HealpixFitsWarning: No NSIDE in the header file : will use length of array
  warnings.warn("No NSIDE in the header file : will use length of array", HealpixFitsWarning)


NSIDE is ok, but it is not healpy format
colnames = ['HPIX', 'VALUE']
Assuming celestial coordinates. If the coordinates are galactic, use gal2eq()


/Users/jackelvinpoole/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in divide


corr2pt using bin_slop = 0.1
corr2pt: auto


In [ ]:
w_dict